In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [7]:
data = pd.read_csv('https://raw.githubusercontent.com/prathameshks/ML-From-Scratch/main/MNIST_DATA/train.csv')

In [8]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [9]:
Y_train

array([4, 7, 8, ..., 1, 4, 8], dtype=int64)

In [10]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [13]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [7]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[6 6 8 ... 2 6 5] [3 9 1 ... 0 8 4]
0.10419512195121951
Iteration:  10
[2 2 8 ... 8 4 5] [3 9 1 ... 0 8 4]
0.13795121951219513
Iteration:  20
[2 2 1 ... 2 4 4] [3 9 1 ... 0 8 4]
0.18565853658536585
Iteration:  30
[2 4 1 ... 2 4 4] [3 9 1 ... 0 8 4]
0.2508292682926829
Iteration:  40
[7 9 1 ... 2 4 4] [3 9 1 ... 0 8 4]
0.31809756097560976
Iteration:  50
[7 9 1 ... 2 6 4] [3 9 1 ... 0 8 4]
0.377390243902439
Iteration:  60
[7 9 1 ... 2 6 4] [3 9 1 ... 0 8 4]
0.4241219512195122
Iteration:  70
[2 9 1 ... 0 6 4] [3 9 1 ... 0 8 4]
0.46463414634146344
Iteration:  80
[2 9 1 ... 0 5 4] [3 9 1 ... 0 8 4]
0.5035121951219512
Iteration:  90
[2 9 1 ... 0 5 4] [3 9 1 ... 0 8 4]
0.5393658536585366
Iteration:  100
[8 9 1 ... 0 5 4] [3 9 1 ... 0 8 4]
0.5711219512195121
Iteration:  110
[3 9 1 ... 0 5 4] [3 9 1 ... 0 8 4]
0.6006829268292683
Iteration:  120
[3 9 1 ... 0 5 4] [3 9 1 ... 0 8 4]
0.6252926829268293
Iteration:  130
[3 9 1 ... 0 5 4] [3 9 1 ... 0 8 4]
0.6465853658536586
Iteration:  1

In [3]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [7]:
test_prediction(10100, W1, b1, W2, b2)

NameError: name 'X_train' is not defined

In [17]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[0 2 2 9 6 9 0 9 7 0 5 6 6 1 8 4 1 3 2 4 7 2 4 5 3 8 0 5 1 8 4 8 0 2 7 1 4
 9 8 2 7 7 6 6 3 7 8 7 8 3 7 3 9 8 5 2 6 2 2 3 8 6 2 5 8 2 5 6 3 0 7 9 9 1
 8 0 8 2 7 9 6 4 8 1 2 0 7 5 7 3 0 5 6 5 0 8 9 2 0 1 9 0 0 8 4 4 6 2 0 0 6
 5 8 7 5 7 6 1 6 4 2 7 4 2 3 9 8 6 4 8 0 4 2 6 5 3 8 4 2 3 2 1 9 3 3 5 0 1
 6 8 9 1 5 9 6 1 8 8 2 2 5 9 9 0 3 1 6 7 6 2 8 6 4 0 0 9 3 4 9 3 5 9 4 9 0
 6 0 2 5 8 7 3 6 6 9 5 1 4 1 4 7 0 7 3 6 3 1 3 9 8 1 7 8 7 6 9 5 6 0 5 3 9
 9 7 0 6 5 7 1 8 1 3 2 5 3 0 4 0 2 2 2 9 3 1 1 5 3 2 8 7 7 0 5 0 7 6 9 7 0
 9 4 7 9 2 6 2 3 6 2 8 3 9 1 2 8 0 2 7 9 5 8 5 0 9 4 0 7 6 7 3 3 3 4 3 6 6
 5 2 6 5 2 9 3 3 1 8 4 3 1 1 7 6 9 2 4 7 9 0 3 3 9 7 1 6 6 2 6 4 8 7 1 8 6
 5 9 7 0 5 2 1 0 0 4 9 7 3 7 8 1 1 9 6 4 2 4 1 5 9 8 5 7 0 4 5 2 8 2 1 2 2
 2 0 7 1 5 2 5 1 2 0 9 1 6 1 8 2 5 8 5 5 4 9 2 9 4 6 1 6 5 4 7 6 0 1 8 4 4
 0 4 1 9 4 3 1 4 9 7 0 7 1 3 2 3 9 4 4 3 9 4 7 3 2 1 3 0 7 2 9 8 2 5 4 2 1
 5 9 1 2 4 3 3 6 8 9 5 1 4 7 1 9 4 6 0 6 0 6 4 0 2 9 2 6 0 0 2 8 8 8 8 3 2
 6 5 7 5 8 0 6 6 9 1 8 3 

0.839

In [11]:
# save model
model = (W1, b1, W2, b2)
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [6]:
# load model
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
    W1, b1, W2, b2 = model
